# Step 1 : Installing and Importing Libraries

In [322]:
!pip3 install -r requirements.txt

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [268]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import random
import json
import string
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
from pylab import rcParams
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import plotly.express as px
from tqdm import tqdm

# Step 2: Defining Constants

In [150]:
BASE_URL = "https://www.aljazeera.com"
MOZ_URL= BASE_URL+"/where/mozambique/"
ARTICLE_FILE = 'articles.json'
#n = number of articles to scrape
n=10

# STEP 3: Web Scrapping to get n Articles

In [276]:
#function to get links of n articles from MOZ_URL
def get_n_article_links(n):
    req = requests.get(MOZ_URL)
    soup = BeautifulSoup(req.text, "html.parser")
    links = []
    
    #after inspecting the web page we get the info that links are under tag 'a' and class 'u-clickable-card__link'
    card_links=soup.findAll('a',{'class':'u-clickable-card__link'})
    for link in tqdm(card_links[:10]):
        links.append(BASE_URL+link.get('href'))
    return links

In [277]:
#function to get articles from the links
def get_articles(links):
    articles=[]
    for link in tqdm(links):
        req = requests.get(link)
        soup = BeautifulSoup(req.text, "html.parser")
        
        #title
        title = soup.find('title').text
        #removing extra data from end of title
        title = title[:title.index('|')]
        
        #date
        date_div = soup.find('div',{'class':'article-dates'})
        date_text= date_div.findAll('span')[1].text
        
        #images and image captions
        images=[]
        figures= soup.findAll('figure')
        for fig in figures:
            image = fig.find('img')
            if image:
                images.append({'url':BASE_URL+image['src'],'caption':fig.text})
        
        
        #main article
        paras=soup.find('main').findAll('p')
        text=''
        for para in paras:
            text+=para.text+'\n'
            
        article = {'title':title,
                   'date':date_text,
                   'images':images,
                   'text':text
                  }
        articles.append(article)
        
    return articles

In [278]:
#function to save dictionary to file as JSON
def save_dict_to_json(data):
    with open(ARTICLE_FILE, "a") as outfile:
        for d in tqdm(data):
            json.dump(d, outfile,indent=4)
            outfile.write('\n')

In [279]:
#1. Get links for n articles from given URL
links=get_n_article_links(n)

#2. Fetch articles using the links
articles=get_articles(links)

#3. save articles to file 
save_dict_to_json(articles)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 5400.16it/s]


# 4. Preprocessing Data

In [319]:
def preprocessing(articles):
    new_data=[]
    for i in tqdm(range(len(articles))):
        article=articles[i]
        
        #1. Converting to lower case
        article['title']=article['title'].lower()
        article['text']=article['text'].lower()
        print(len(article['text'].split()))
        
        #2. Removing stopwords
        text_tokens = word_tokenize(article['text'])
        tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
        article['text']=(" ").join(tokens_without_sw)
        
    return articles

In [320]:
articles=preprocessing(articles)

  0%|                                                                                                                                                                                | 0/10 [00:00<?, ?it/s]

282


 10%|████████████████▊                                                                                                                                                       | 1/10 [00:00<00:06,  1.41it/s]

121


 20%|█████████████████████████████████▌                                                                                                                                      | 2/10 [00:01<00:03,  2.14it/s]

...

 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                 | 8/10 [00:06<00:01,  1.24it/s]

183


 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                | 9/10 [00:06<00:00,  1.39it/s]

377


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.31it/s]


# 5. Analysis using SentimentIntensityAnalyzer

In [300]:
title_with_sent=[]
analyser = SentimentIntensityAnalyzer()
for i,article in enumerate(articles):
    analysis = analyser.polarity_scores(article['title']+' '+article['text'])
    title_with_sent.append({'title':article['title'], 'compound':analysis['compound']})

In [321]:
fig = px.bar(title_with_sent, x='title', y='compound')
fig.show()

# 6. Analysis using TextBlob

In [302]:
df = pd.DataFrame(articles)

In [303]:
df=df.drop(columns=['date','images'])
df.head

<bound method NDFrame.head of                                                title  \
0  floods hit south africa’s kwazulu-natal provin...   
1  mozambique: cyclone gombe death toll rises to 53    
2  mozambique announces new prime minister after ...   
3  analysis: can african gas replace russian supp...   
4  dozens dead from tropical storm ana in souther...   
5  southern africa bloc sadc extends mozambique m...   
6                         climate change and famine    
7  in mozambique, kagame says rwandan troops’ wor...   
8  rwanda, mozambique forces recapture port city ...   
9  rwanda deploys 1,000 soldiers to mozambique’s ...   

                                                text  
0  floods happening month torrential rains killed...  
1  recent years , southern africa suffered repeat...  
2  new ministers named major reshuffle president ...  
3  lack infrastructure , capacity , could hurt co...  
4  least 70 people killed storm struck madagascar...  
5  regional body says p

In [304]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [305]:
#adding columns for subjectivity and polarity
df['subjectivity']=(df['title']+' '+df['text']).apply(getSubjectivity)
df['polarity']=(df['title']+' '+df['text']).apply(getPolarity)

df

,title,text,subjectivity,polarity
0,floods hit south africa’s kwazulu-natal provin...,floods happening month torrential rains killed...,0.375397,0.013889
1,mozambique: cyclone gombe death toll rises to 53,"recent years , southern africa suffered repeat...",0.304762,-0.155952
2,mozambique announces new prime minister after ...,new ministers named major reshuffle president ...,0.468182,0.126010
3,analysis: can african gas replace russian supp...,"lack infrastructure , capacity , could hurt co...",0.352018,0.066648
4,dozens dead from tropical storm ana in souther...,least 70 people killed storm struck madagascar...,0.326087,-0.032609
5,southern africa bloc sadc extends mozambique m...,regional body says progress made rebels cabo d...,0.280729,0.071354
6,climate change and famine,madagascar brink famine worst drought 40 years...,0.550000,-0.500000
7,"in mozambique, kagame says rwandan troops’ wor...",rwandan president says country ’ force help se...,0.298150,0.036586
8,"rwanda, mozambique forces recapture port city ...","mocimboa praia , home africa ’ biggest natural...",0.307407,0.081250
9,"rwanda deploys 1,000 soldiers to mozambique’s ...",government kigali says deployment request moza...,0.361656,-0.006560


In [306]:
#function to compute +ve, -ve or neutral analysis from polarity
def get_analysis(score):
    if score >0:
        return 'positive'
    elif score == 0:
        return 'neutral'
    else:
        return 'negative'

In [307]:
df['analysis'] = df['polarity'].apply(get_analysis)

In [316]:
#scatter plot between polarity and subjectivity

fig = px.scatter(df,x='polarity',y='subjectivity',color='analysis',hover_data=["title"])
fig.update_traces(marker_size=10)
fig.show()